In [4]:
# t_pre_clean.py (for t_news.jsonl)
import json, re, unicodedata
from pathlib import Path
from pythainlp.util import normalize

# ---------- PATH ----------
input_file = Path("data/t_news.jsonl")       # ← ใช้ไฟล์ข่าวที่ได้จาก prepare_data.py
output_file = Path("data/cleaned_news.jsonl")
output_file.parent.mkdir(parents=True, exist_ok=True)

# ---------- ฟังก์ชันตรวจความถูกต้อง ----------
def is_valid(text: str) -> bool:
    """กรองข่าวที่ไม่เหมาะกับ labeling"""
    if len(text) < 80:
        return False
    # ถ้ามี URL เยอะ แปลว่ายังไม่ได้ clean
    if text.count("http") > 1:
        return False
    # ถ้าเป็นภาษาไทยน้อยกว่า 30% ให้ตัดออก
    th_chars = len(re.findall(r"[\u0E00-\u0E7F]", text))
    if th_chars / max(len(text), 1) < 0.3:
        return False
    return True

# ---------- ฟังก์ชัน clean ----------
def clean_text(txt: str) -> str:
    """ล้าง HTML, อีโมจิ, ขยะท้ายบทความ โดยคงเครื่องหมายที่ช่วยตัดคำ"""
    if not txt:
        return ""

    # 1) เอาแท็ก HTML ออก
    txt = re.sub(r"<[^>]+>", " ", txt)

    # 2) ลบ emoji และสัญลักษณ์พิเศษนอกช่วง Unicode ไทย/อังกฤษ
    txt = re.sub(r"[\U00010000-\U0010ffff]", "", txt)

    # 3) ลบส่วนเกินที่มักเจอในเว็บข่าว
    txt = re.sub(r"พิมพ์ แชร์เรื่องนี้ แชร์เรื่องนี้ Line Twitter Facebook คัดลอกลิงก์ - ก ก", "", txt)
    txt = re.sub(r"(อ่านต่อที่.*|คลิกชมภาพ.*|ดูเพิ่มเติม.*|เครดิตภาพ.*)", "", txt)
    txt = re.sub(r"(appeared first on .*|The post .*)", "", txt, flags=re.I)
    txt = re.sub(r"(Facebook.*?Twitter.*?LINE)", "", txt)
    txt = re.sub(r"&#82\d{2};", "", txt)
    txt = re.sub(r"\s*\[\]\s*", " ", txt)
    txt = re.sub(r"อ่านข่าวต้นฉบับ:.*$", "", txt)
    txt = re.sub(r"Thairath Online Thairath Money Thairath Shopping Thairath Plus Thairath TV MIRROR หน้าแรก ฟุตบอลต่างประเทศ พรีเมียร์ลีก ยูฟ่า แชมเปียนส์ลีก บุนเดสลีกา กัลโช เซเรีย อา ลา ลีกา เจลีก ลีกอื่นๆ ตารางคะแนน", "", txt)
    txt = re.sub(r"โปรแกรม/ผลการแข่งขัน ดาวซัลโว ฟุตบอลไทย ฟุตบอลทีมชาติไทย ไทยลีก ฟุตซอลไทย Carabao 7-a-Side Cup ตารางคะแนน โปรแกรม/ผลการแข่งขัน คอลัมน์ บี บางปะกง สนามกีฬาแห่งชาติ ไฟต์สปอร์ต มวยไทย มวยโลก กีฬาโลก ", "", txt)
    txt = re.sub(r"วอลเลย์บอล แบดมินตัน มอเตอร์สปอร์ต กีฬาอื่นๆ วิดีโอ ไฮไลต์ เรื่องรอบขอบสนาม ไทยรัฐเล่ากีฬา sport daily เชียร์ไทยให้กึกก้อง แกลเลอรี่ หน้าแรก ฟุตบอลต่างประเทศ พรีเมียร์ลีก ยูฟ่า แชมเปียนส์ลีก บุนเดสลีกา กัลโช เซเรีย อา ลา ลีกา เจลีก ลีกอื่นๆ ตารางคะแนน ", "", txt)
    txt = re.sub(r"วอลเลย์บอล แบดมินตัน มอเตอร์สปอร์ต กีฬาอื่นๆ วิดีโอ ไฮไลต์ เรื่องรอบขอบสนาม ไทยรัฐเล่ากีฬา sport daily เชียร์ไทยให้กึกก้อง แกลเลอรี่ ติดตามเราได้ที่ THAIRATH ON", "", txt)
    
    # 4) normalize ตัวอักษร (เช่น พิมพ์ซ้อน / วรรณยุกต์เพี้ยน)
    txt = unicodedata.normalize("NFC", txt)
    txt = normalize(txt)

    # 5) ลบช่องว่างซ้ำ
    txt = re.sub(r"\s+", " ", txt).strip()

    # 6) คงเครื่องหมายช่วย segmentation เช่น จุด / วงเล็บ / เครื่องหมายคำพูด
    txt = re.sub(r"([?!])", r" \1 ", txt)
    txt = re.sub(r"([()\"“”‘’])", r" \1 ", txt)
    txt = re.sub(r"\s+", " ", txt).strip()

    return txt


# ---------- MAIN ----------
def main():
    print("🧹 เริ่มทำความสะอาดข่าวจาก:", input_file)
    total, kept = 0, 0

    with input_file.open(encoding="utf-8") as fin, output_file.open("w", encoding="utf-8") as fout:
        for line in fin:
            total += 1
            try:
                item = json.loads(line)
            except json.JSONDecodeError:
                continue

            text = clean_text(item.get("text", ""))
            if is_valid(text):
                fout.write(json.dumps({
                    "title": item.get("title", "").strip(),
                    "link": item.get("link", ""),
                    "text": text
                }, ensure_ascii=False) + "\n")
                kept += 1

            if kept % 50 == 0 and kept > 0:
                print(f"   ✅ คลีนแล้ว {kept} ข่าว (จาก {total})")

    print(f"\n🎯 เสร็จสิ้น! คลีนข่าวทั้งหมด {kept}/{total} บทความ → {output_file}")


if __name__ == "__main__":
    main()


🧹 เริ่มทำความสะอาดข่าวจาก: data\t_news.jsonl


FileNotFoundError: [Errno 2] No such file or directory: 'data\\t_news.jsonl'

In [52]:
# t_pre_clean_v2.py
import json, re, unicodedata
from pathlib import Path
from pythainlp.util import normalize

# ---------- PATH ----------
input_file = Path("data/cleaned_news.jsonl")   # ไฟล์จากขั้นตอนก่อนหน้า
output_file = Path("data/ready_for_label.jsonl")
output_file.parent.mkdir(parents=True, exist_ok=True)

# ---------- ฟังก์ชันช่วย ----------
def remove_tail_noise(text: str) -> str:
    """
    ลบข้อความส่วนท้ายที่ไม่ใช่เนื้อข่าว เช่น เครดิต อัลบั้ม แชร์เรื่องนี้ ฯลฯ
    """
    patterns = [
        r"โหลดเพิ่ม.*", r"ดูทั้งหมด\s*\d+\s*ภาพ.*", r"แชร์เรื่องนี้.*",
        r"ขอขอบคุณ.*", r"ผู้เขียน.*", r"Facebook.*", r"Twitter.*",
        r"LINE.*", r"ติดตามโซเชียล.*", r"เม้าท์กันทั้งเมือง.*"
    ]
    for p in patterns:
        text = re.split(p, text)[0]
    return text.strip()

def clean_text(txt: str) -> str:
    """ทำความสะอาดข้อความก่อน labeling"""
    if not txt:
        return ""
    # ลบ HTML tag และ emoji
    txt = re.sub(r"<[^>]+>", " ", txt)
    txt = re.sub(r"[\U00010000-\U0010ffff]", "", txt)

    # Normalize ตัวอักษร (วรรณยุกต์, พิมพ์ซ้อน)
    txt = unicodedata.normalize("NFC", txt)
    txt = normalize(txt)

    # ลบเครื่องหมายคำพูดซ้ำๆ และช่องว่างเกิน
    txt = re.sub(r"[\"“”‘’]+", "", txt)
    txt = re.sub(r"\s{2,}", " ", txt)
    
    # CLEAN FUCKING STUPID WORDS
    txt = re.sub(" พิมพ์", " ", txt)
    txt = re.sub("&nbsp;", " ", txt)
    txt = re.sub(r"\s*\+\s*", " ", txt)

    txt = re.sub(r"TAGS:.*$", "", txt, flags=re.MULTILINE)
    txt = re.sub(r"แหล่งอ้างอิง.*$", "", txt, flags=re.MULTILINE)
    txt = re.sub(r"อ้างอิงบางส่วน:.*$", "", txt, flags=re.MULTILINE)
    txt = re.sub(r"อ้างอิง .*$", "", txt, flags=re.MULTILINE)

    # ลบเครื่องหมายตกแต่ง ( - ก ก + ) ที่เจอในเว็บข่าว
    txt = re.sub(r"-\s*ก\s*ก\s*\+", "", txt)

    # ลบ timestamp, วันที่ในวงเล็บ
    txt = re.sub(r"\d{1,2}\s*[ก-ฮ]+\.\s*\d{2,4}", "", txt)
    txt = re.sub(r"\(.*?น\.\)", "", txt)

    # ลบส่วนท้ายที่ไม่ใช่เนื้อข่าว
    txt = remove_tail_noise(txt)

    # ลบ space ซ้ำอีกครั้ง
    txt = re.sub(r"\s+", " ", txt).strip()

    return txt

def is_valid(text: str) -> bool:
    """เช็กว่าข่าวเหมาะสมสำหรับ labeling หรือไม่"""
    if len(text) < 100:
        return False
    th_chars = len(re.findall(r"[\u0E00-\u0E7F]", text))
    if th_chars / max(len(text), 1) < 0.4:
        return False
    return True


# ---------- MAIN ----------
def main():
    print("🧼 เริ่มทำความสะอาดขั้นสุดท้าย เพื่อเตรียม labeling ...")
    total, kept = 0, 0

    with input_file.open(encoding="utf-8") as fin, output_file.open("w", encoding="utf-8") as fout:
        for line in fin:
            total += 1
            try:
                item = json.loads(line)
            except json.JSONDecodeError:
                continue

            text = clean_text(item.get("text", ""))
            if is_valid(text):
                fout.write(json.dumps({
                    "title": item.get("title", "").strip(),
                    "text": text
                }, ensure_ascii=False) + "\n")
                kept += 1

            if kept % 50 == 0 and kept > 0:
                print(f"   ✅ คลีนแล้ว {kept} ข่าว (จาก {total})")

    print(f"\n🎯 เสร็จสิ้น! เตรียมข่าวพร้อม labeling ได้ทั้งหมด {kept}/{total} ข่าว → {output_file}")


if __name__ == "__main__":
    main()


🧼 เริ่มทำความสะอาดขั้นสุดท้าย เพื่อเตรียม labeling ...
   ✅ คลีนแล้ว 50 ข่าว (จาก 50)
   ✅ คลีนแล้ว 100 ข่าว (จาก 100)
   ✅ คลีนแล้ว 150 ข่าว (จาก 150)
   ✅ คลีนแล้ว 200 ข่าว (จาก 220)

🎯 เสร็จสิ้น! เตรียมข่าวพร้อม labeling ได้ทั้งหมด 206/226 ข่าว → data\ready_for_label.jsonl
